In [ ]:
## you want to firstly run !pip instal optuna

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import KFold,cross_val_score
from sklearn.ensemble import RandomForestRegressor
import optuna
import plotly.express as px

c:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## setup the dataset

X = load_diabetes(as_frame = True)['data']
y = load_diabetes()['target']


In [3]:
# step 1: set up an objective function

def objective(trial):
    
    # define the hyperparameters
    n_estimators = trial.suggest_int('n_estimators',10, 300)
    max_depth = trial.suggest_int('max_depth', 2, 20)
    criterion = trial.suggest_categorical('criterion', 
                                          ['squared_error', 'absolute_error', 
                                           'friedman_mse', 'poisson'])
    
    regressor = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth,
                                      criterion=criterion, random_state=23)
    
    score = -cross_val_score(estimator=regressor, X=X, y=y,
                             scoring='neg_root_mean_squared_error',cv = 5).mean()
    
    return score

# step 2: create the study
study = optuna.create_study(direction='minimize',
                            sampler = optuna.samplers.RandomSampler(seed=23))

# step 3: optimize the study
study.optimize(objective, n_trials=50,n_jobs = 1)

[I 2024-12-19 14:31:24,684] A new study created in memory with name: no-name-25c9264c-1004-48cb-a784-95698f72d4f9
[I 2024-12-19 14:31:47,476] Trial 0 finished with value: 58.010616267741625 and parameters: {'n_estimators': 160, 'max_depth': 19, 'criterion': 'squared_error'}. Best is trial 0 with value: 58.010616267741625.
[I 2024-12-19 14:31:58,087] Trial 1 finished with value: 57.73487706540742 and parameters: {'n_estimators': 58, 'max_depth': 9, 'criterion': 'poisson'}. Best is trial 1 with value: 57.73487706540742.
[I 2024-12-19 14:33:18,104] Trial 2 finished with value: 57.75895199544866 and parameters: {'n_estimators': 267, 'max_depth': 7, 'criterion': 'absolute_error'}. Best is trial 1 with value: 57.73487706540742.
[I 2024-12-19 14:33:32,030] Trial 3 finished with value: 57.826393589528415 and parameters: {'n_estimators': 95, 'max_depth': 7, 'criterion': 'squared_error'}. Best is trial 1 with value: 57.73487706540742.
[I 2024-12-19 14:33:43,684] Trial 4 finished with value: 57.1

In [4]:
study.best_params

{'n_estimators': 277, 'max_depth': 4, 'criterion': 'poisson'}

In [5]:
study.best_value

57.10106685465706

In [ ]:
optuna.visualization.plot_param_importances(study)


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_slice(study)

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)